In [8]:
import face_recognition as fr
import os
import cv2
import numpy as np
from time import sleep

    looks through the faces folder and encodes all
    the faces

    :return: dict of (name, image encoded)

In [9]:
cap = cv2.VideoCapture(0)

In [10]:
def get_encoded_faces():

    encoded = {}

    for dirpath, dnames, fnames in os.walk("./faces"):
        for f in fnames:
            if f.endswith(".jpg") or f.endswith(".png"):
                face = fr.load_image_file("faces/" + f)
                encoding = fr.face_encodings(face)[0]
                encoded[f.split(".")[0]] = encoding

    return encoded


    encode a face given the file name

In [11]:
def unknown_image_encoded(img):

    face = fr.load_image_file("faces/" + img)
    encoding = fr.face_encodings(face)[0]

    return encoding

    will find all of the faces in a given image and label
    them if it knows what they are

    :param im: str of file path
    :return: list of face names

In [12]:
def classify_face(frame):
    faces = get_encoded_faces()
    faces_encoded = list(faces.values())
    known_face_names = list(faces.keys())

    face_locations = fr.face_locations(frame)
    unknown_face_encodings = fr.face_encodings(frame, face_locations)

    face_names = []
    for face_encoding in unknown_face_encodings:
        matches = fr.compare_faces(faces_encoded, face_encoding)
        name = "Unknown"

        face_distances = fr.face_distance(faces_encoded, face_encoding)
        best_match_index = np.argmin(face_distances)
        if matches[best_match_index]:
            name = known_face_names[best_match_index]

        face_names.append(name)

    return face_names

In [13]:
def cam():
    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()
        if ret:
            face_names = classify_face(frame)
            for (top, right, bottom, left), name in zip(fr.face_locations(frame), face_names):
                cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
                cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
                font = cv2.FONT_HERSHEY_DUPLEX
                cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

            cv2.imshow("Preview", frame)
            if cv2.waitKey(1) == 27: # Presionar la tecla Esc para detener la vista previa
                break
    cap.release()
    cv2.destroyAllWindows()


In [14]:
cam()

KeyboardInterrupt: 